# Notebook to play with the 1999 Republic Referendum data

Thanks to Zach A for repeatedly nudging me to do this

Thanks to David Michael for the data

## Libraries

In [1]:
import pandas as pd

## Create file by polling place

The input file here is by polling place by electorate

I want a file just by polling place - for mapping et al

In [3]:
filepath = '1999_referenda_output/republic_referendum_by_electorate_by_polling_place.csv'

df_republic = pd.read_csv(
    filepath
)

df_republic.head(3)

,state,electorate,polling_place,polling_place_raw,yes_or_no,yes_n,yes_p,no_n,no_p,formal_n,formal_p,informal_n,informal_p,total_n,total_p
0,SA,Adelaide,Adelaide East,Adelaide East,Yes,465,0.6700,229,0.3300,694,0.9914,6,0.0086,700,0.0086
1,SA,Adelaide,Adelaide Hospital,Adelaide Hospital,Yes,187,0.6172,116,0.3828,303,0.9806,6,0.0194,309,0.0038
2,SA,Adelaide,Adelaide South,Adelaide South,Yes,1056,0.6467,577,0.3533,1633,0.9903,16,0.0097,1649,0.0202
